In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.CNN import convNN as conv
import tensorflow as tf



path = "/media/jahan/solo/"
os.chdir(path)

labels = pd.read_csv("CheXpert-v1.0-small/train.csv")
print(len(labels))
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
print(len(labels))


all_image_paths = labels.Path.values

labels = [1 if x == 1 else 0 for x in labels["No Finding"]]

img_path = all_image_paths[0]
print(img_path)

img_raw = tf.io.read_file(img_path)
print(repr(img_raw)[:100]+"...")
img_tensor = tf.image.decode_jpeg(img_raw,channels=1)
print(img_tensor.shape)
print(img_tensor.dtype)


def preprocess_image(image):
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [256, 256])
    image /= 255.0  # normalize to [0,1] range
    return image

def load_and_preprocess_image(path): # load from path and return tensor
    image = tf.io.read_file(path)
    return preprocess_image(image)

import matplotlib.pyplot as plt

image_path = all_image_paths[0]
label = labels[0]

plt.imshow(load_and_preprocess_image(img_path).numpy())
plt.grid(False)
print()

W0813 19:53:02.644839 139737634625344 deprecation_wrapper.py:119] From /home/jahan/Documents/CXR_CNN/algorithms/autoencoder.py:11: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



223414
191027
CheXpert-v1.0-small/train/patient00001/study1/view1_frontal.jpg
<tf.Tensor: id=1, shape=(), dtype=string, numpy=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x...
(320, 389, 1)
<dtype: 'uint8'>



In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.CNN import convNN as conv
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot

path = "/media/jahan/solo/"
os.chdir(path)

labels = pd.read_csv("CheXpert-v1.0-small/train.csv")
print(len(labels))
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
print(len(labels))


all_image_paths = labels.Path.values

labels = [1 if x == 1 else 0 for x in labels["No Finding"]]
encode = onehot()

labels = encode.fit_transform(np.array(labels).reshape((-1,1))).toarray()
print(labels[:5])

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


save_path = "/home/jahan/Documents/CXR_CNN/output/"


resnet = conv(image_paths=all_image_paths, labels=labels, save_path=save_path)
resnet.build_dataset(batch_size=32)
resnet.build()

W0813 20:48:22.643981 140102162777920 deprecation_wrapper.py:119] From /home/jahan/Documents/CXR_CNN/algorithms/autoencoder.py:11: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



223414
191027
[[0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]]


/home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 16) 416         input_1[0][0]                    
__________________________________________________________________________________________________
12c (BatchNormalization)        (None, 256, 256, 16) 64          conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           12c[0][0]                        
______________________________________________________________________________________________

In [2]:
resnet.train(5)

Epoch 1/5


W0813 20:48:34.276372 140102162777920 deprecation.py:323] From /home/jahan/miniconda3/envs/cxr/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


5000/5000 [==============================] - 1240s 248ms/step - loss: 0.2393 - auc: 0.9630 - false_negatives: 13768.0000 - false_positives: 13768.0000 - true_negatives: 146232.0000 - true_positives: 146232.0000
Epoch 2/5
5000/5000 [==============================] - 1153s 231ms/step - loss: 0.2467 - auc: 0.9618 - false_negatives: 15007.0000 - false_positives: 15007.0000 - true_negatives: 144980.0000 - true_positives: 144980.0000
Epoch 3/5
5000/5000 [==============================] - 743s 149ms/step - loss: 0.2368 - auc: 0.9647 - false_negatives: 14372.0000 - false_positives: 14372.0000 - true_negatives: 145615.0000 - true_positives: 145615.0000
Epoch 4/5
5000/5000 [==============================] - 698s 140ms/step - loss: 0.2256 - auc: 0.9677 - false_negatives: 13601.0000 - false_positives: 13601.0000 - true_negatives: 146386.0000 - true_positives: 146386.0000
Epoch 5/5
5000/5000 [==============================] - 701s 140ms/step - loss: 0.2244 - auc: 0.9681 - false_negatives: 13634.000

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.CNN import convNN as conv
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot


path = "/media/jahan/solo/"
os.chdir(path)
save_path = "/home/jahan/Documents/CXR_CNN/output/"

labels = pd.read_csv("CheXpert-v1.0-small/train.csv")
print(len(labels))
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
print(len(labels))

labels = pd.read_csv("CheXpert-v1.0-small/valid.csv")
print(len(labels))
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
print(len(labels))

resnet = conv(image_paths=None, labels=None, save_path=save_path)
resnet.build(load=True)

test_image_paths = labels.Path.values

labels = np.array([1 if x == 1 else 0 for x in labels["No Finding"]])
encode = onehot()

encoded = encode.fit_transform(labels.reshape((-1,1))).toarray()
print(encoded[:5])

metrics = resnet.predict(test_image_paths,encoded)

W0814 05:33:53.363185 140207692703552 deprecation_wrapper.py:119] From /home/jahan/Documents/CXR_CNN/algorithms/autoencoder.py:11: The name tf.enable_eager_execution is deprecated. Please use tf.compat.v1.enable_eager_execution instead.



223414
191027
234
202
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 256, 256, 16) 416         input_1[0][0]                    
__________________________________________________________________________________________________
12c (BatchNormalization)        (None, 256, 256, 16) 64          conv1[0][0]                      
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           12c[0][0]                        
________________________________________________________________________

W0814 05:33:56.104208 140207692703552 deprecation.py:323] From /home/jahan/miniconda3/envs/ml_cpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
/home/jahan/miniconda3/envs/ml_cpu/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[[1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]
7/7 [==============================] - 7s 1s/step - loss: 0.2740 - auc: 0.9679 - false_negatives: 23.0000 - false_positives: 23.0000 - true_negatives: 179.0000 - true_positives: 179.0000


In [5]:
from sklearn.metrics import roc_auc_score as auc

print(auc(encoded, metrics[1]))
print(metrics)
print(encoded)

0.9208916083916083
[[0.2740052895886557, 0.96789527, 23.0, 23.0, 179.0, 179.0], array([[0.8511346 , 0.14886545],
       [0.40698776, 0.5930122 ],
       [0.9288566 , 0.07114343],
       [0.43405095, 0.5659491 ],
       [0.94491696, 0.05508302],
       [0.9479595 , 0.05204051],
       [0.6492452 , 0.35075477],
       [0.49785712, 0.5021428 ],
       [0.941888  , 0.05811204],
       [0.6524254 , 0.34757462],
       [0.6746581 , 0.32534188],
       [0.6447982 , 0.35520175],
       [0.9563245 , 0.0436755 ],
       [0.7600686 , 0.23993139],
       [0.5347908 , 0.46520913],
       [0.98693895, 0.013061  ],
       [0.51525515, 0.48474488],
       [0.87541324, 0.1245868 ],
       [0.51310545, 0.48689458],
       [0.5598005 , 0.44019955],
       [0.8312715 , 0.16872844],
       [0.64630026, 0.35369974],
       [0.5977482 , 0.40225175],
       [0.57623994, 0.42376006],
       [0.77261066, 0.22738935],
       [0.88004744, 0.1199525 ],
       [0.69331646, 0.30668357],
       [0.6572283 , 0.3427717